# Интеграция CoreML RusTorch - Python Bindings

Этот блокнот демонстрирует, как использовать функциональность CoreML RusTorch через Python bindings.

## Настройка и импорты

In [ ]:
# Импорт Python bindings RusTorch
try:
    import rustorch
    print(f"✅ Версия RusTorch: {rustorch.__version__}")
    print(f"📝 Описание: {rustorch.__description__}")
    print(f"👥 Автор: {rustorch.__author__}")
except ImportError as e:
    print(f"❌ Не удалось импортировать RusTorch: {e}")
    print("Пожалуйста, соберите с помощью maturin develop")
    exit(1)

import numpy as np
import platform

print(f"🖥️ Платформа: {platform.system()} {platform.release()}")
print(f"🐍 Версия Python: {platform.python_version()}")

## Проверка доступности CoreML

In [ ]:
# Проверка функциональности CoreML
try:
    # Проверить, доступен ли CoreML
    coreml_available = rustorch.is_coreml_available()
    print(f"🍎 CoreML доступен: {coreml_available}")
    
    if coreml_available:
        print("🎉 CoreML доступен!")
        
        # Получить информацию об устройстве
        device_info = rustorch.get_coreml_device_info()
        print("📱 Информация об устройстве CoreML:")
        print(device_info)
    else:
        print("⚠️ CoreML недоступен")
        if platform.system() != "Darwin":
            print("CoreML доступен только на macOS")
        else:
            print("Функции CoreML могут быть не включены")
            
except AttributeError:
    print("❌ Функции CoreML не найдены")
    print("Возможно, не собрано с функциями CoreML")
    coreml_available = False
except Exception as e:
    print(f"❌ Ошибка при проверке CoreML: {e}")
    coreml_available = False

## Создание устройства CoreML и операции

In [ ]:
if coreml_available:
    try:
        # Создать устройство CoreML
        device = rustorch.CoreMLDevice(device_id=0)
        print(f"🖥️ Устройство CoreML создано: {device}")
        
        # Получить информацию об устройстве
        print(f"🆔 ID устройства: {device.device_id()}")
        print(f"✅ Доступно: {device.is_available()}")
        print(f"💾 Лимит памяти: {device.memory_limit()} байт")
        print(f"🧮 Лимит вычислительных блоков: {device.compute_units_limit()}")
        print(f"📚 Размер кэша модели: {device.model_cache_size()}")
        
        # Очистка кэша
        device.cleanup_cache()
        print("🧹 Кэш очищен")
        
    except Exception as e:
        print(f"❌ Ошибка операции устройства CoreML: {e}")
else:
    print("⚠️ Пропускаем операции устройства, так как CoreML недоступен")

## Конфигурация бэкенда CoreML

In [ ]:
if coreml_available:
    try:
        # Создать конфигурацию бэкенда CoreML
        config = rustorch.CoreMLBackendConfig(
            enable_caching=True,
            max_cache_size=200,
            enable_profiling=True,
            auto_fallback=True
        )
        print(f"⚙️ Конфигурация бэкенда: {config}")
        
        # Проверить и изменить значения конфигурации
        print(f"📊 Включить кэширование: {config.enable_caching}")
        print(f"🗂️ Максимальный размер кэша: {config.max_cache_size}")
        print(f"📈 Включить профилирование: {config.enable_profiling}")
        print(f"🔄 Автоматический откат: {config.auto_fallback}")
        
        # Изменить конфигурацию
        config.enable_profiling = False
        config.max_cache_size = 150
        print(f"\n🔧 Обновленная конфигурация: {config}")
        
        # Создать бэкенд CoreML
        backend = rustorch.CoreMLBackend(config)
        print(f"🚀 Бэкенд CoreML: {backend}")
        print(f"✅ Бэкенд доступен: {backend.is_available()}")
        
        # Получить статистику бэкенда
        stats = backend.get_stats()
        print(f"📊 Статистика бэкенда: {stats}")
        print(f"   Всего операций: {stats.total_operations}")
        print(f"   Попадания в кэш: {stats.cache_hits}")
        print(f"   Промахи кэша: {stats.cache_misses}")
        print(f"   Операции отката: {stats.fallback_operations}")
        print(f"   Процент попаданий в кэш: {stats.cache_hit_rate():.2%}")
        print(f"   Процент откатов: {stats.fallback_rate():.2%}")
        print(f"   Среднее время выполнения: {stats.average_execution_time_ms:.2f}мс")
        
        # Очистка кэша
        backend.cleanup_cache()
        print("\n🧹 Кэш бэкенда очищен")
        
    except Exception as e:
        print(f"❌ Ошибка операции бэкенда CoreML: {e}")
else:
    print("⚠️ Пропускаем операции бэкенда, так как CoreML недоступен")

## Базовые операции с тензорами (CPU)

Для сравнения с CoreML сначала выполним базовые операции на CPU.

In [ ]:
try:
    # Создание базовых тензоров и операции
    print("🧮 Базовые операции с тензорами (CPU)")
    
    # Создать тензоры из массивов NumPy (упрощенный интерфейс)
    data_a = np.random.randn(2, 3).astype(np.float32)
    data_b = np.random.randn(3, 2).astype(np.float32)
    
    print(f"📐 Форма матрицы A: {data_a.shape}")
    print(f"📐 Форма матрицы B: {data_b.shape}")
    
    # Умножение матриц с NumPy (для сравнения)
    numpy_result = np.matmul(data_a, data_b)
    print(f"✅ Форма результата matmul NumPy: {numpy_result.shape}")
    print(f"📊 Результат (первые элементы): {numpy_result.flatten()[:4]}")
    
    print("\n🚀 Операции CPU завершены")
    
except Exception as e:
    print(f"❌ Ошибка операции с тензором: {e}")

## Симуляция сравнения производительности

In [ ]:
import time

def benchmark_matrix_operations():
    """Сравнить производительность с разными размерами матриц"""
    
    sizes = [(64, 64), (128, 128), (256, 256), (512, 512)]
    
    print("🏁 Сравнение производительности:")
    print("Размер\t\tВремя CPU (мс)\tОжидаемый CoreML (мс)")
    print("-" * 58)
    
    for size in sizes:
        # Измерить время выполнения CPU
        a = np.random.randn(*size).astype(np.float32)
        b = np.random.randn(size[1], size[0]).astype(np.float32)
        
        start_time = time.time()
        result = np.matmul(a, b)
        cpu_time = (time.time() - start_time) * 1000
        
        # Ожидаемое время CoreML (гипотетическое)
        # В реальной реализации использовать фактические измерения бэкенда CoreML
        expected_coreml_time = cpu_time * 0.6  # Предположение: CoreML на 40% быстрее
        
        print(f"{size[0]}x{size[1]}\t\t{cpu_time:.2f}\t\t{expected_coreml_time:.2f}")

benchmark_matrix_operations()

print("\n📝 Примечание: Времена CoreML гипотетические. Реальные значения зависят от конкретной реализации.")

## Симуляция выбора устройства

In [ ]:
def simulate_device_selection():
    """Симулировать умный выбор устройства"""
    
    operations = [
        ("Умножение маленькой матрицы", (16, 16), "CPU"),
        ("Умножение средней матрицы", (128, 128), "Metal GPU"),
        ("Умножение большой матрицы", (512, 512), "CoreML" if coreml_available else "Metal GPU"),
        ("Функция активации", (32, 64, 128, 128), "Metal GPU"),
        ("Маленькая свертка", (1, 3, 32, 32), "CPU"),
        ("Большая свертка", (16, 64, 224, 224), "CoreML" if coreml_available else "Metal GPU"),
        ("Операции с комплексными числами", (128, 128), "Metal GPU"),  # CoreML не поддерживается
        ("Статистическое распределение", (1000,), "CPU"),  # CoreML не поддерживается
    ]
    
    print("🎯 Симуляция умного выбора устройства:")
    print("Операция\t\t\tФорма тензора\t\tВыбранное устройство")
    print("-" * 78)
    
    for name, shape, device in operations:
        shape_str = "x".join(map(str, shape))
        print(f"{name:<31}\t{shape_str:<15}\t{device}")
    
    print("\n📝 Логика выбора:")
    print("  • Маленькие операции: CPU (избежать накладных расходов)")
    print("  • Средние операции: Metal GPU (сбалансированный)")
    print("  • Большие операции: CoreML (оптимизированный)")
    print("  • Неподдерживаемые операции: откат GPU/CPU")

simulate_device_selection()

## Практический пример: простой слой нейронной сети

In [ ]:
def simulate_neural_network_layer():
    """Симулировать слой нейронной сети"""
    
    print("🧠 Симуляция слоя нейронной сети:")
    
    # Размер партии и конфигурация слоя
    batch_size = 32
    input_features = 784  # 28x28 MNIST
    hidden_features = 256
    output_features = 10  # 10 классов
    
    print(f"📊 Размер партии: {batch_size}")
    print(f"🔢 Входные признаки: {input_features}")
    print(f"🧮 Скрытые признаки: {hidden_features}")
    print(f"🎯 Выходные признаки: {output_features}")
    
    # Симуляция прямого прохода
    steps = [
        ("Вход → Скрытый", f"({batch_size}, {input_features}) @ ({input_features}, {hidden_features})", "CoreML" if coreml_available else "Metal"),
        ("Активация ReLU", f"({batch_size}, {hidden_features})", "Metal"),
        ("Скрытый → Выход", f"({batch_size}, {hidden_features}) @ ({hidden_features}, {output_features})", "CoreML" if coreml_available else "Metal"),
        ("Softmax", f"({batch_size}, {output_features})", "CPU"),
    ]
    
    print("\n🔄 Симуляция прямого прохода:")
    total_time = 0
    
    for step, shape, device in steps:
        # Виртуальное время выполнения (мс)
        if device == "CoreML":
            time_ms = np.random.uniform(0.5, 2.0)
        elif device == "Metal":
            time_ms = np.random.uniform(1.0, 3.0)
        else:  # CPU
            time_ms = np.random.uniform(0.2, 1.0)
        
        total_time += time_ms
        print(f"  {step:<15} {shape:<30} {device:<8} {time_ms:.2f}мс")
    
    print(f"\n⏱️ Общее время прямого прохода: {total_time:.2f}мс")
    print(f"🚀 Расчетная пропускная способность: {1000/total_time:.0f} партий/секунду")

simulate_neural_network_layer()

## Резюме и следующие шаги

In [ ]:
print("📋 Резюме интеграции CoreML RusTorch:")
print()
print("✅ Завершенные элементы:")
print("  • Настройка среды Jupyter")
print("  • Создание ядра Rust и Python bindings")
print("  • Проверка доступности CoreML")
print("  • Управление устройствами и конфигурация")
print("  • Статистика и профилирование бэкенда")
print("  • Умный выбор устройства")
print()
print("🚧 Будущее развитие:")
print("  • Реальная реализация операций CoreML")
print("  • Бенчмаркинг производительности")
print("  • Больше функций активации и типов слоев")
print("  • Улучшения обработки ошибок")
print("  • Оптимизация памяти")
print()
print("🎯 Рекомендуемые следующие шаги:")
print("  1. Загрузить и протестировать реальные модели CoreML")
print("  2. Сравнить производительность Metal и CoreML")
print("  3. Протестировать с реальными рабочими процессами глубокого обучения")
print("  4. Оценить в производственной среде")

if coreml_available:
    print("\n🎉 Поздравляем! CoreML доступен и все функции можно протестировать.")
else:
    print("\n⚠️ CoreML недоступен, но базовые функции работают.")
    print("   Рекомендуем собрать с включенными функциями CoreML на macOS.")